In [10]:
import requests
from bs4 import BeautifulSoup
# import pandas as pd
from urllib.parse import urljoin
import json

In [11]:
base_url="https://www.cdc.gov/"
visited_urls=set()
urls_to_visit=[base_url]
max_documents=5
number_of_documents=0
scraped_data=[]

In [12]:
exclude=[
            "section",  "for-authors", 
            "editorial-board", "about", "issues", 
            "posts", "toggle", "view=desktop","search"
        ]
include= [
    
                "healthcare", "health", "hospital", "clinic", "emergency", "primary care",
                "disease", "infection", "symptoms", "diagnosis", "treatment", "therapy",
                "chronic", "acute", "mental health", "cancer", "diabetes", "cardiology",
                "clinical trial", "meta-analysis", "systematic review", "case study",
                "epidemiology", "randomized controlled trial", "biostatistics",
                "nutrition", "exercise", "stress", "prevention", "self-care", "fitness",
                "vaccination", "immunization", "screening",
                
                "health policy", "public health", "health reform", "universal healthcare",
                "insurance", "health equity", "pandemic", "quarantine", "outbreak"
            ]


In [13]:
while urls_to_visit and len(visited_urls) < max_documents:
    current_url=urls_to_visit.pop(0)
    if current_url in visited_urls:
        continue
    
    try:
        print(f"{current_url}")
        response=requests.get(current_url,timeout=10)
        print(response.status_code)
        if response.status_code!=200:
            continue

        # print(response.text)
        print(type(response.text))
        soup=BeautifulSoup(response.text,"html.parser")
        print(type(soup))
        # print(soup)
        
        
        title=soup.title.text
        paragraphs=soup.find_all('p')
        # print(paragraphs)
        content="".join([p.text for  p in paragraphs])
        # print(content)
        
        # print(title)
        if (keyword in content for keyword in include):
            print(f"It includes the word")
        scraped_data.append({'url':current_url,'title':title,'content':content})
        visited_urls.add(current_url)
        number_of_documents+=1
        
        for link in soup.find_all('a',href=True):
            abosulteUrl=urljoin(current_url,link['href'])
            if abosulteUrl not in visited_urls and abosulteUrl.startswith("http") and not any(keyword in abosulteUrl for keyword in exclude):
                urls_to_visit.append(abosulteUrl)
                
    except Exception as e:
        print(f"Error crawling {current_url}: {e}")

    

https://www.cdc.gov/
200
<class 'str'>
<class 'bs4.BeautifulSoup'>
It includes the word
https://www.cdc.gov/#content
200
<class 'str'>
<class 'bs4.BeautifulSoup'>
It includes the word
https://www.cdc.gov/#headerSearch
200
<class 'str'>
<class 'bs4.BeautifulSoup'>
It includes the word
https://www.cdc.gov/#gov-notice
200
<class 'str'>
<class 'bs4.BeautifulSoup'>
It includes the word
https://www.cdc.gov/spanish/
200
<class 'str'>
<class 'bs4.BeautifulSoup'>
It includes the word


In [14]:
print(number_of_documents)
if scraped_data:
    with open('healthcaredata.json', 'w', encoding='utf-8') as f:
        json.dump(scraped_data, f, ensure_ascii=False, indent=4)
    print("Data saved to healthcaredata.json")
else:
    print("No data was scraped.")

5
Data saved to healthcaredata.json
